In [ ]:
import pandas as pd
import numpy as np

In [ ]:
train = pd.read_csv('train.csv')
valid = pd.read_csv('valid.csv')
test = pd.read_csv('test.csv')

In [ ]:
train.columns

Index(['time_period', 'season', 'ccg_nm', 'adn_dgnm', 'sex_cd', 'age_group',
       'trns_cnt', 'ttl_trns_amt', '업종 대분류'],
      dtype='object')

In [ ]:
X_train = train[['time_period', 'season', 'ccg_nm', 'adn_dgnm', 'sex_cd', 'age_group',
       'trns_cnt', 'ttl_trns_amt']]
y_train = train['업종 대분류']

In [ ]:
X_valid = valid[['time_period', 'season', 'ccg_nm', 'adn_dgnm', 'sex_cd', 'age_group',
       'trns_cnt', 'ttl_trns_amt']]
y_valid = valid['업종 대분류']

X_test = test[['time_period', 'season', 'ccg_nm', 'adn_dgnm', 'sex_cd', 'age_group',
       'trns_cnt', 'ttl_trns_amt']]
y_test = test['업종 대분류']

In [ ]:
import xgboost as xgb

In [ ]:
from sklearn.preprocessing import LabelEncoder

# LabelEncoder 객체 생성
label_encoder = LabelEncoder()
#  y_train 라벨 인코딩
y_train_encoded = label_encoder.fit_transform(y_train)
y_valid_encoded = label_encoder.fit_transform(y_valid)
y_test_encoded = label_encoder.fit_transform(y_test)
# 인코딩된 값 확인
print(y_train_encoded)

[4 4 0 ... 4 3 3]


In [ ]:
model_xgb_no_rw  = xgb.XGBClassifier(random_state=42,
                              # scale_pos_weight=ratio,
                            #   scale_pos_weight=recall_81['scale_pos_weight'],
                            #   max_depth = recall_81['max_depth'],
                            #   n_estimators= recall_81['n_estimators'],
                            #   alpha = recall_81['alpha'],
                            #   gamma = recall_81['gamma'],
                            #   learning_rate = recall_81['learning_rate'],
                             )

model_xgb_no_rw .fit(X_train, y_train_encoded)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [ ]:
test_predict = model_xgb_no_rw.predict(X_test)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# 폰트 = applegothic
plt.rcParams['font.family'] = 'NanumGothic'

In [ ]:
import pandas as pd
from sklearn.metrics import classification_report

# Classification report 생성
report_dict = classification_report(y_test_encoded, test_predict, target_names=label_encoder.classes_, output_dict=True)

# DataFrame으로 변환
report_df = pd.DataFrame(report_dict).transpose()

# 결과 출력
print(report_df)

              precision    recall  f1-score       support
양식             0.907530  0.875345  0.891147   4709.000000
일식             0.632310  0.457672  0.531001   1890.000000
주점             0.534929  0.585249  0.558959   1871.000000
중식             0.653553  0.668542  0.660963   2311.000000
한식             0.796380  0.868707  0.830973   5065.000000
accuracy       0.758993  0.758993  0.758993      0.758993
macro avg      0.704940  0.691103  0.694608  15846.000000
weighted avg   0.758141  0.758993  0.756164  15846.000000


In [ ]:
X_train

,time_period,season,ccg_nm,adn_dgnm,sex_cd,age_group,trns_cnt,ttl_trns_amt
0,0.0,0.0,0.291399,0.230675,1.0,3.0,3,116500
1,2.0,2.0,0.292961,0.282280,1.0,4.0,3698,120356260
2,2.0,0.0,0.344193,0.395249,1.0,2.0,41,354000
3,2.0,0.0,0.285168,0.277022,1.0,8.0,11,291500
4,1.0,1.0,0.297285,0.311144,0.0,4.0,72,241400
...,...,...,...,...,...,...,...,...
50702,2.0,0.0,0.300865,0.223007,0.0,7.0,4,110000
50703,0.0,1.0,0.303952,0.232489,0.0,3.0,1,24500
50704,1.0,3.0,0.267183,0.230068,0.0,5.0,19,294000
50705,2.0,2.0,0.285168,0.327693,1.0,7.0,9,160500


In [ ]:
import pandas as pd

def predict_business_category(input_data, model, label_encoder):
    """
    새로운 입력 데이터를 받아 업종 대분류명을 예측하는 함수.

    Parameters:
        input_data (dict): 예측할 데이터 (딕셔너리 형식).
        model: 학습된 XGBoost 모델.
        label_encoder: 학습에 사용된 LabelEncoder.

    Returns:
        str: 예측된 업종 대분류명.
    """
    # 딕셔너리 데이터를 DataFrame으로 변환
    input_df = pd.DataFrame([input_data])

    # 예측 수행
    y_pred = model.predict(input_df)

    # 예측된 값을 원래 클래스명으로 변환
    predicted_label = label_encoder.inverse_transform(y_pred)[0]

    return predicted_label

# 예시 입력 데이터 (딕셔너리 형식)
example_input = {
    "time_period": 0.0	,
    "season": 0.0,
    "ccg_nm": 0.291399,
    "adn_dgnm": 0.230675,
    "sex_cd": 1.0,
    "age_group": 3.0,
    "trns_cnt": 3,
    "ttl_trns_amt": 116500
}

# 예측 수행
predicted_category = predict_business_category(example_input, model_xgb_no_rw, label_encoder)

# 결과 출력
print(f"예측된 업종 대분류명: {predicted_category}")


예측된 업종 대분류명: 한식


In [ ]:
!pip install konlpy

In [ ]:
from konlpy.tag import Okt
import pandas as pd
import random

In [ ]:
df = pd.read_csv('name.csv')

sampled_df = df.groupby('업종 대분류').apply(lambda x: x.sample(n=10000, random_state=42))

# 인덱스 리셋 (기존의 그룹 인덱스를 제거)
sampled_df = sampled_df.reset_index(drop=True)

df = sampled_df

import random
import pickle
from konlpy.tag import Okt
from concurrent.futures import ProcessPoolExecutor

okt = Okt()

<ipython-input-53-3f2dc6937e9b>:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sampled_df = df.groupby('업종 대분류').apply(lambda x: x.sample(n=10000, random_state=42))


In [ ]:
def extract_nouns(df, business_type):
    # 선택된 업장 종류의 가게 명에서 명사 추출
    business_names = df[df['업종 대분류'] == business_type]['사업장명'].tolist()
    nouns = []
    for name in business_names:
        nouns.extend([word for word, pos in okt.pos(name) if pos == 'Noun'])
    return nouns

def generate_new_name(nouns):
    # 명사 조합으로 새로운 가게 명 생성
    if len(nouns) >= 2:
        return random.choice(nouns) + random.choice(nouns)
    elif nouns:
        return nouns[0] + "샵"  # 예비 대체명
    else:
        return "추천할 가게 명이 없습니다."

# 예: '카페' 업장 종류의 가게 명 추천
business_type = predicted_category
nouns = extract_nouns(df, predicted_category)
print("추천 가게 명:", generate_new_name(nouns))

추천 가게 명: 정식갈비
